In [ ]:
# 실행 방법
# $ python manage.py shell_plus --notebook

In [140]:
! pip install requests
! pip install pandas
! pip install numpy
! pip install matplotlib
! pip install datetime

You should consider upgrading via the 'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import requests
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request
from io import BytesIO

In [148]:
import requests
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request
from io import BytesIO

my_api_key = '6a5ece7778e61cb35c55c953b8743b0d'

movies, english_movies = [], []
for idx in range(1, 501):
    # Data 정리하기
    popular_movie_url = 'https://api.themoviedb.org/3/movie/popular?api_key=' + my_api_key + '&language=ko-kr&page=' + str(idx)

    res = requests.get(popular_movie_url).text
    data = json.loads(res)
    movies += data['results']
    
# (-----------------------------------------------------------------------------)

    english_movie_url = 'https://api.themoviedb.org/3/movie/popular?api_key=' + my_api_key + '&language=en-US&page=' + str(idx)

    english_res = requests.get(english_movie_url).text
    english_data = json.loads(english_res)
    english_movies += english_data['results']
    
features_name = list(movies[0].keys())

df_movies = pd.DataFrame(movies, columns= features_name)

english_features_name = list(english_movies[0].keys())

df_english_movies = pd.DataFrame(english_movies, columns= english_features_name)

df_movies['eng_title'] = df_english_movies['title']
df_movies = df_movies.drop(columns=['original_title'])

# 장르 가져오기 https://api.themoviedb.org/3/genre/movie/list?api_key=<<api_key>>
genre_url = 'https://api.themoviedb.org/3/genre/movie/list?api_key=' + my_api_key
genre_res = requests.get(genre_url).text
genre_list = json.loads(genre_res)['genres']
df_genres = pd.DataFrame(genre_list, columns=['id', 'name'])

    
genre_names = []
for idx in range(len(df_movies)):
    movie = df_movies.iloc[idx]
    change_list = []
    for gen in movie['genre_ids']:
        change_list.append(list(df_genres['name'][df_genres['id'] == gen])[0])
    genre_names.append(change_list)
df_movies['genre_ids'] = genre_names

total_yt_url_list = []

for movie_idx in range(len(df_movies)):
    movie_id = df_movies.iloc[movie_idx]['id']
    movie_id = str(movie_id)
    video_url = 'https://api.themoviedb.org/3/movie/' + movie_id + '/videos?api_key=' + my_api_key +'&language=en-US'
    video_res = requests.get(video_url).text
    video_data = json.loads(video_res)
    video_data = video_data['results'][:5]

    yt_url_list = []

    for video_key in video_data:
        video_key = video_key['key']
        yt_url = 'https://www.youtube.com/watch?v=' + video_key
        yt_url_list.append(yt_url)
    total_yt_url_list.append(yt_url_list)
df_movies['video'] = total_yt_url_list

# 1:N table을 위한 id, video에 대한 df를 생성
df_all_video = df_movies[['id','video']]
df_all_video = df_all_video.explode("video", ignore_index=True)
df_all_video.rename(columns = {'id' : 'movie_id'}, inplace = True)

# 1:N table을 위한 movie_id, genre_ids에 대한 df를 생성
df_all_genres = df_movies[['movie_id','genre_ids']]
df_all_genres = df_all_genres.explode("genre_ids", ignore_index=True)
df_movies = df_movies.drop(columns = ['genre_ids'])

# video를 지우고 column명을 id에서 movie_id로 변환
df_movies = df_movies.drop(columns = ['video'])
df_movies.rename(columns = {'id' : 'movie_id'}, inplace = True)


df_movies.to_json(r'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\movies\fixtures\all_movie.json', force_ascii=False, orient = 'records', indent=4)
df_all_video.to_json(r'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\movies\fixtures\all_movie_video.json', force_ascii=False, orient = 'records', indent=4)
df_all_genres.to_json(r'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\movies\fixtures\all_movie_genre.json', force_ascii=False, orient = 'records', indent=4)

# 데이터 형태 바꾸기 (django model에 추가하기 위한 과정)

with open('./movies/fixtures/AllMovie.json') as f:
    all_movie_js = json.loads(f.read()) ## json 라이브러리 이용

with open('./movies/fixtures/all_movie_video.json') as f:
    all_video_js = json.loads(f.read()) ## json 라이브러리 이용

with open('./movies/fixtures/all_movie_genre.json') as f:
    all_genre_js = json.loads(f.read()) ## json 라이브러리 이용

all_movie_list = list(all_movie_js)
all_video_list = list(all_video_js)
all_genre_list = list(all_genre_js)

for movie in all_movie_list:
    movie['backdrop_path'] = 'https://image.tmdb.org/t/p/original' + movie['backdrop_path']
    movie['poster_path'] = 'https://image.tmdb.org/t/p/original' + movie['poster_path']

new_all_movie_list, new_all_video_list, new_all_genre_list = [], [], []
for movie in all_movie_list:
    # Movie 모델 필드명에 맞추어 데이터를 저장함.
    data = {
        "model": "movies.AllMovie",
        'fields': {
            "adult": movie['adult'],
            "backdrop_path": movie['backdrop_path'],
            "movie_id": movie['movie_id'],
            "original_language": movie['original_language'],
            "overview": movie['overview'],
            "popularity": movie['popularity'],
            "poster_path": movie['poster_path'],
            "release_date": movie['release_date'],
            "title": movie['title'],
            "vote_average": movie['vote_average'],
            "vote_count": movie['vote_count'],
            "eng_title": movie['eng_title']
        },
    }
    new_all_movie_list.append(data)
    
for video in all_video_list:
    # Movie 모델 필드명에 맞추어 데이터를 저장함.
    data = {
        "model": "movies.AllRelatedVideo",
        'fields': {
            "movie_id" : video['movie_id'],
            "video" : video['video']
        },
    }
    new_all_video_list.append(data)

for genre in all_genre_list:
    # Movie 모델 필드명에 맞추어 데이터를 저장함.
    data = {
        "model": "movies.AllGenre",
        'fields': {
            "movie_id" : genre['movie_id'],
            "genre_ids" : genre['genre_ids']
        },
    }
    new_all_genre_list.append(data)

with open('./movies/fixtures/all_movie.json','w') as f:
    json.dump(new_all_movie_list,f, ensure_ascii=False, indent=4)
with open('./movies/fixtures/all_movie_video.json','w') as f:
    json.dump(new_all_video_list,f, ensure_ascii=False, indent=4)
with open('./movies/fixtures/all_movie_genre.json','w') as f:
    json.dump(new_all_genre_list,f, ensure_ascii=False, indent=4)

,adult,backdrop_path,genre_ids,id,original_language,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,eng_title
0,False,/aAgGrfBwna1nO4M2USxwFgK5O0t.jpg,"[Horror, Thriller, Fantasy]",713704,en,길바닥에서 지친 베스가 비좁은 아파트에서 세 아이를 키우는 엘리를 뒤늦게 찾아간다....,5030.612,/5ik4ATKmNtmJU6AYD0bLm56BCVM.jpg,2023-04-12,이블 데드 라이즈,"[https://www.youtube.com/watch?v=2wc5VpRc450, ...",7.1,1017,Evil Dead Rise
1,False,/3CxUndGhUcZdt1Zggjdb2HkLLQX.jpg,"[Action, Adventure, Science Fiction]",640146,en,"슈퍼히어로 파트너인 스캇 랭과 호프 반 다인, 호프의 부모 재닛 반 다인과 행크 핌...",3023.011,/sz6mTIDDQmR3DYgJudiTmoW2gR5.jpg,2023-02-15,앤트맨과 와스프: 퀀텀매니아,"[https://www.youtube.com/watch?v=UJZx8MayWxk, ...",6.5,2525,Ant-Man and the Wasp: Quantumania
2,False,/nLBRD7UPR6GjmWQp6ASAfCTaWKX.jpg,"[Animation, Family, Adventure, Fantasy, Comedy]",502356,en,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,4675.159,/dlGyzCxbBQK1U2O5o31Z1hB6erc.jpg,2023-04-05,슈퍼 마리오 브라더스,"[https://www.youtube.com/watch?v=LTFGH0rJ-EY, ...",7.5,2202,The Super Mario Bros. Movie
3,False,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,"[Horror, Thriller, Fantasy]",758323,en,바티칸이 인정한 공식 수석 엑소시스트이자 최고의 구마사제 '가브리엘 아모르트’ 신부...,2378.680,/fA1xRPplBqW87wWh0hO1pwbQmX8.jpg,2023-04-05,엑소시스트: 더 바티칸,"[https://www.youtube.com/watch?v=R16ysWgt6oM, ...",7.2,971,The Pope's Exorcist
4,False,/99NBEpQEM4uLItZY2RquqdqdSXM.jpg,"[Science Fiction, Adventure, Action]",447365,en,가모라를 잃고 슬픔에 빠져 있던 피터 퀼이 위기에 처한 은하계와 동료를 지키기 위해...,2137.214,/b7epV2cQZVIR4u5VZraDwD0AgiE.jpg,2023-05-03,가디언즈 오브 갤럭시: Volume 3,"[https://www.youtube.com/watch?v=c8cZzltEH2Q, ...",8.2,1200,Guardians of the Galaxy Vol. 3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,False,/8vqQs9VMBECtDJJNnS0ZgVGTLCQ.jpg,"[Romance, Drama]",86835,en,성공한 작가 릭(크리스찬 베일)은 어린 시절 아버지가 들려준 왕자의 이야기를 기억한...,10.027,/nuUWNIDfFtrSd3jWNs65hsbkMq.jpg,2015-02-08,나이트 오브 컵스,[https://www.youtube.com/watch?v=vl6BWTIZeOg],5.6,573,Knight of Cups
9996,False,/3NhTdtnIwkZPbT5rAQ1yzItx2aH.jpg,"[Horror, Science Fiction, Thriller]",12476,en,미국의 L. A 지역. 지금까지 선량한 시민이던 데브리즈(Jack Devries: ...,11.022,/pCoFtULUIebq13EU9glcZjfdC1a.jpg,1987-10-30,히든,"[https://www.youtube.com/watch?v=ofcsHJ6sZM0, ...",6.9,330,The Hidden
9997,False,/EicRiXUPvkkQRPOOn2oNZpNwnu.jpg,"[Drama, Thriller]",8653,ko,"저수지의 낚시터, 낚시 가게 주인인 희진(서정)은 낚시꾼들에게 안주나 먹을 것들을 ...",8.023,/fqubdg1F7XTYgmDm84LA45plzvK.jpg,2000-04-22,섬,[https://www.youtube.com/watch?v=NjpAjs70S9Y],7.0,271,The Isle
9998,False,/oJIve4VPGXt9dpa2WcKOBeFb782.jpg,[Drama],445456,en,,11.531,/lZOzJBELeg0c24msieggqWVvjB1.jpg,2018-03-01,55 스텝스,[https://www.youtube.com/watch?v=i0m2Tlbb43c],6.8,143,55 Steps


In [20]:
with open('./movies/fixtures/AllMovie.json') as f:
    all_movie_js = json.loads(f.read()) ## json 라이브러리 이용
    
all_movie_list = list(all_movie_js)

new_all_movie_list = []

for movie in all_movie_list:
    movie['backdrop_path'] = 'https://image.tmdb.org/t/p/original' + movie['backdrop_path']
    movie['poster_path'] = 'https://image.tmdb.org/t/p/original' + movie['poster_path']

for movie in all_movie_list:
    # Movie 모델 필드명에 맞추어 데이터를 저장함.
    data = {
        "model": "movies.AllMovie",
        'fields': {
            "adult": movie['adult'],
            "backdrop_path": movie['backdrop_path'],
            "movie_id": movie['movie_id'],
            "original_language": movie['original_language'],
            "overview": movie['overview'],
            "popularity": movie['popularity'],
            "poster_path": movie['poster_path'],
            "release_date": movie['release_date'],
            "title": movie['title'],
            "vote_average": movie['vote_average'],
            "vote_count": movie['vote_count'],
            "eng_title": movie['eng_title']
        },
    }
    new_all_movie_list.append(data)

with open('./movies/fixtures/all_movie.json','w') as f:
    json.dump(new_all_movie_list,f, ensure_ascii=False, indent=4)

In [3]:
import json, pandas as pd
 
with open('./movies/fixtures/AllMovie.json') as f:
    all_movie_js = json.loads(f.read()) ## json 라이브러리 이용

with open('./movies/fixtures/all_movie_video.json') as f:
    all_video_js = json.loads(f.read()) ## json 라이브러리 이용

with open('./movies/fixtures/all_movie_genre.json') as f:
    all_genre_js = json.loads(f.read()) ## json 라이브러리 이용

all_movie_list = list(all_movie_js)
all_video_list = list(all_video_js)
all_genre_list = list(all_genre_js)

new_all_movie_list, new_all_video_list, new_all_genre_list = [], [], []
for movie in all_movie_list:
    # Movie 모델 필드명에 맞추어 데이터를 저장함.
    data = {
        "model": "movies.AllMovie",
        'fields': {
            "adult": movie['adult'],
            "backdrop_path": movie['backdrop_path'],
            "movie_id": movie['movie_id'],
            "original_language": movie['original_language'],
            "overview": movie['overview'],
            "popularity": movie['popularity'],
            "poster_path": 'https://image.tmdb.org/t/p/original/' + movie['poster_path'],
            "release_date": movie['release_date'],
            "title": movie['title'],
            "vote_average": movie['vote_average'],
            "vote_count": movie['vote_count'],
            "eng_title": movie['eng_title']
        },
    }
    new_all_movie_list.append(data)
    
for video in all_video_list:
    # Movie 모델 필드명에 맞추어 데이터를 저장함.
    data = {
        "model": "movies.AllRelatedVideo",
        'fields': {
            "movie_id" : video['movie_id'],
            "video" : video['video']
        },
    }
    new_all_video_list.append(data)

for genre in all_genre_list:
    # Movie 모델 필드명에 맞추어 데이터를 저장함.
    data = {
        "model": "movies.AllGenre",
        'fields': {
            "movie_id" : genre['movie_id'],
            "genre_ids" : genre['genre_ids']
        },
    }
    new_all_genre_list.append(data)

with open('./movies/fixtures/all_movie.json','w') as f:
    json.dump(new_all_movie_list,f, ensure_ascii=False, indent=4)
with open('./movies/fixtures/all_movie_video.json','w') as f:
    json.dump(new_all_video_list,f, ensure_ascii=False, indent=4)
with open('./movies/fixtures/all_movie_genre.json','w') as f:
    json.dump(new_all_genre_list,f, ensure_ascii=False, indent=4)

KeyError: 'movie_id'

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request
from io import BytesIO
from datetime import date, datetime, timedelta

def get_today_movie_list():
    # 오늘 날짜 자동 계산
    ymd = date.today() - timedelta(1)
    ymd = datetime.strftime(ymd, '%Y%m%d')
    # print(ymd)

    # 오늘 상영중인 영화
    my_api_key = '6a5ece7778e61cb35c55c953b8743b0d'
    today_movie_key = '0163c18e093080d6b1ad11fa2fadfad1'

    today = ymd

    today_movie_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=' + today_movie_key + '&targetDt=' + today
    today_movie_res = requests.get(today_movie_url).text
    today_movie_data = json.loads(today_movie_res)
    today_movies = today_movie_data['boxOfficeResult']['dailyBoxOfficeList']

    # 하루에 한번
    # 오늘의 영화 제목을 활용하여 movie detail을 불러오기
    today_movie_detail_list = []
    english_movies = []
    for movie in today_movies:
        movie_name = movie['movieNm']
        search_movie_url = 'https://api.themoviedb.org/3/search/movie?api_key=' + my_api_key + '&query=' + movie_name + '&include_adult=false&language=ko-KR&page=1'

        search_movie_res = requests.get(search_movie_url).text
        search_movie_data = json.loads(search_movie_res)
        search_movie = search_movie_data['results'][0]
        
        today_movie_detail_list.append(search_movie)
    
        english_movie_url = 'https://api.themoviedb.org/3/search/movie?api_key=' + my_api_key + '&query=' + movie_name + '&include_adult=false&language=en-US&page=1'

        english_res = requests.get(english_movie_url).text
        english_data = json.loads(english_res)
        english_movies += english_data['results']

    df_today_movies = pd.DataFrame(today_movie_detail_list, columns = today_movie_detail_list[0].keys())

    english_features_name = list(english_movies[0].keys())

    df_english_movies = pd.DataFrame(english_movies, columns= english_features_name)

    df_today_movies['eng_title'] = df_english_movies['title']
    df_today_movies = df_today_movies.drop(columns=['original_title'])

    # 장르 변환
    genre_url = 'https://api.themoviedb.org/3/genre/movie/list?api_key=' + my_api_key
    genre_res = requests.get(genre_url).text
    genre_list = json.loads(genre_res)['genres']
    df_genres = pd.DataFrame(genre_list, columns=['id', 'name'])


    today_genre_names = []
    for idx in range(len(df_today_movies)):
        today_movie = df_today_movies.iloc[idx]
        change_list = []
        for gen in today_movie['genre_ids']:
            change_list.append(list(df_genres['name'][df_genres['id'] == gen])[0])
        today_genre_names.append(change_list)
    df_today_movies['genre_ids'] = today_genre_names

    # poster_path
    image_url_list = []
    for idx in range(len(df_today_movies)):
        today_movie = df_today_movies.iloc[idx]
        image_url = 'https://image.tmdb.org/t/p/original' + df_today_movies.iloc[idx]['poster_path']
        image_url_list.append(image_url)
    
    df_today_movies['poster_path'] = image_url_list
    
        
    # df_today_movies = df_today_movies.drop(columns = ['video'])

    # 유튜브 video url
    # 'https://www.youtube.com/watch?v=' + video에 있는 key

    # 연관 video url

    total_yt_url_list = []

    for today_movie_idx in range(len(df_today_movies)):
        movie_id = df_today_movies.iloc[today_movie_idx]['id']
        movie_id = str(movie_id)
        video_url = 'https://api.themoviedb.org/3/movie/' + movie_id + '/videos?api_key=' + my_api_key +'&language=en-US'
        video_res = requests.get(video_url).text
        video_data = json.loads(video_res)
        video_data = video_data['results'][:5]

        yt_url_list = []

        for video_key in video_data:
            video_key = video_key['key']
            yt_url = 'https://www.youtube.com/watch?v=' + video_key
            yt_url_list.append(yt_url)
        total_yt_url_list.append(yt_url_list)
    df_today_movies['video'] = total_yt_url_list
    
    # 1:N을 위한 video 따로 저장
    df_today_video = df_today_movies[['id','video']]
    df_today_video = df_today_video.explode("video", ignore_index=True)
    df_today_video.rename(columns = {'id' : 'movie_id'}, inplace = True)
    
    # video column을 삭제하고 id column을 movie_id로 변환
    df_today_movies = df_today_movies.drop(columns = ['video'])
    df_today_movies.rename(columns = {'id' : 'movie_id'}, inplace = True)
    
    # 1:N table을 위한 movie_id, genre_ids에 대한 df를 생성
    df_today_genres = df_today_movies[['movie_id','genre_ids']]
    df_today_genres = df_today_genres.explode("genre_ids", ignore_index=True)
    df_today_movies = df_today_movies.drop(columns = ['genre_ids'])
    
    
    df_today_movies.to_json(r'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\movies\fixtures\{}_movie.json'.format(today), force_ascii=False, orient = 'records', indent=4)
    df_today_video.to_json(r'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\movies\fixtures\{}_video.json'.format(today), force_ascii=False, orient = 'records', indent=4)
    df_today_genres.to_json(r'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\movies\fixtures\{}_genre.json'.format(today), force_ascii=False, orient = 'records', indent=4)
    
 
    with open('./movies/fixtures/{}_movie.json'.format(today)) as f:
        today_movie_js = json.loads(f.read()) ## json 라이브러리 이용

    with open('./movies/fixtures/{}_video.json'.format(today)) as f:
        today_video_js = json.loads(f.read()) ## json 라이브러리 이용

    with open('./movies/fixtures/{}_genre.json'.format(today)) as f:
        today_genre_js = json.loads(f.read()) ## json 라이브러리 이용

    today_movie_list = list(today_movie_js)
    today_video_list = list(today_video_js)
    today_genre_list = list(today_genre_js)

    for movie in today_movie_list:
        if movie['backdrop_path']:
            movie['backdrop_path'] = 'https://image.tmdb.org/t/p/original' + movie['backdrop_path']
    
    new_today_movie_list, new_today_video_list, new_today_genre_list = [], [], []
    for movie in today_movie_list:
        # Movie 모델 필드명에 맞추어 데이터를 저장함.
        data = {
            "model": "movies.TodayMovie",
            'fields': {
                "adult": movie['adult'],
                "backdrop_path": movie['backdrop_path'],
                "movie_id": movie['movie_id'],
                "original_language": movie['original_language'],
                "overview": movie['overview'],
                "popularity": movie['popularity'],
                "poster_path": movie['poster_path'],
                "release_date": movie['release_date'],
                "title": movie['title'],
                "vote_average": movie['vote_average'],
                "vote_count": movie['vote_count'],
                "eng_title": movie['eng_title']
            },
        }
        new_today_movie_list.append(data)
        
    for video in today_video_list:
        # Movie 모델 필드명에 맞추어 데이터를 저장함.
        data = {
            "model": "movies.TodayRelatedVideo",
            'fields': {
                "movie_id" : video['movie_id'],
                "video" : video['video']
            },
        }
        new_today_video_list.append(data)

    for genre in today_genre_list:
        # Movie 모델 필드명에 맞추어 데이터를 저장함.
        data = {
            "model": "movies.TodayGenre",
            'fields': {
                "movie_id" : genre['movie_id'],
                "genre_ids" : genre['genre_ids']
            },
        }
        new_today_genre_list.append(data)

    with open('./movies/fixtures/{}_movie.json'.format(today),'w') as f:
        json.dump(new_today_movie_list,f, ensure_ascii=False, indent=4)
    with open('./movies/fixtures/{}_video.json'.format(today),'w') as f:
        json.dump(new_today_video_list,f, ensure_ascii=False, indent=4)
    with open('./movies/fixtures/{}_genre.json'.format(today),'w') as f:
        json.dump(new_today_genre_list,f, ensure_ascii=False, indent=4)
    
    
    return df_today_movies
get_today_movie_list()

,adult,backdrop_path,movie_id,original_language,overview,popularity,poster_path,release_date,title,vote_average,vote_count,eng_title
0,False,/cEyhk8tZWubni71M6plwLMQFOIX.jpg,385687,en,돔과 그의 패밀리 앞에 나타난 운명의 적 단테. 과거의 그림자는 돔의 모든 것을 파...,1905.026,https://image.tmdb.org/t/p/original/3JT1oE0JIA...,2023-05-17,분노의 질주: 라이드 오어 다이,6.711,83,Fast X
1,False,/5YZbUmjbMa3ClvSW1Wj3D6XGolb.jpg,447365,en,가모라를 잃고 슬픔에 빠져 있던 피터 퀼이 위기에 처한 은하계와 동료를 지키기 위해...,1886.128,https://image.tmdb.org/t/p/original/b7epV2cQZV...,2023-05-03,가디언즈 오브 갤럭시: Volume 3,8.147,1275,Guardians of the Galaxy Vol. 3
2,False,/nLBRD7UPR6GjmWQp6ASAfCTaWKX.jpg,502356,en,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,6545.223,https://image.tmdb.org/t/p/original/dlGyzCxbBQ...,2023-04-05,슈퍼 마리오 브라더스,7.615,2569,The Super Mario Bros. Movie
3,False,/zlHeQg9OTYED2Gm4ZErdvElsUFg.jpg,955666,ja,"어느 날 ‘짱구‘와 동갑내기인 5살 ‘진구’를 데리고, 짱구 가족을 찾아온 수상한 ...",7.073,https://image.tmdb.org/t/p/original/4DKjTt6Vod...,2022-04-22,짱구는 못말려 극장판: 동물소환 닌자 배꼽수비대,7.700,6,Crayon Shin-chan: Mononoke Ninja Chinpūden
4,False,/ceYZCBfwbBwSpGJ6PapNVw5jqLG.jpg,916224,ja,"큐슈의 작은 마을에서 살고 있는 소녀 ‘스즈메’는 어느 날, 여행을 하며 ‘문’을 ...",132.790,https://image.tmdb.org/t/p/original/kVS51ssZF1...,2022-11-11,스즈메의 문단속,7.955,370,Suzume
5,False,/jENycYiYGr5NvMs789zj06JkW5l.jpg,497828,en,호화 크루즈에 협찬으로 승선한 인플루언서 모델 커플. 각양각색의 부자들과 휴가를 즐...,62.457,https://image.tmdb.org/t/p/original/ltEblZr0pw...,2022-09-18,슬픔의 삼각형,7.216,1289,Triangle of Sadness
6,False,None,863138,ko,"과거, 살인자를 검거하는 과정에서 칼에 폐를 찔린 후 장기 이식을 통해 기적적으로 ...",2.938,https://image.tmdb.org/t/p/original/oWnge6xd5i...,2023-04-12,나는 여기에 있다,8.000,1,I Am Here
7,False,/v39H1nUYxQmSphz83VoV4HYjuHt.jpg,636173,ko,"선수 생활 사상 최악의 위기를 맞은 쏘울리스 축구 선수 홍대. 계획도, 의지도 없던...",17.591,https://image.tmdb.org/t/p/original/9RIY2JW0JL...,2023-04-26,드림,6.000,3,Pokémon: Mewtwo Returns
8,False,/h8gHn0OzBoaefsYseUByqsmEDMY.jpg,603692,en,죽을 위기에서 살아난 존 윅은 최고 회의를 쓰러트릴 방법을 찾아낸다. 비로소 완전한...,1100.584,https://image.tmdb.org/t/p/original/9WF6TxCYwd...,2023-03-22,존 윅 4,7.898,1423,Dream
9,False,/3MXFMSRHuRMidknZ5CpFI5oLcY6.jpg,1111319,ko,대한민국 제19대 대통령으로 5년의 임기를 마치고 퇴임한 문재인 전 대통령은 평산마...,5.158,https://image.tmdb.org/t/p/original/e8WjLf0a0o...,2023-05-10,문재인입니다,6.000,1,Lucid Dream


In [25]:
english_movie_url = 'https://api.themoviedb.org/3/movie/popular?api_key=' + my_api_key + '&language=en-US&page=1'

In [26]:
english_res = requests.get(english_movie_url).text
english_data = json.loads(english_res)
english_movies = english_data['results']

In [27]:
popular_movie_url = 'https://api.themoviedb.org/3/movie/popular?api_key=' + my_api_key + '&language=ko-kr&page=1'
# popular_movie_url
# = 'https://api.themoviedb.org/3/movie/popular?api_key=' + my_api_key + '&language=en-US&page=' + page_num
# my_api_key : 내 키
# page_num : 보고싶은 페이지 번호 (total_page : 38302, total_results : 766026)

In [28]:
res = requests.get(popular_movie_url).text
data = json.loads(res)
movies = data['results']

In [29]:
movies

[{'adult': False,
  'backdrop_path': '/aAgGrfBwna1nO4M2USxwFgK5O0t.jpg',
  'genre_ids': [27, 53, 14],
  'id': 713704,
  'original_language': 'en',
  'original_title': 'Evil Dead Rise',
  'overview': '길바닥에서 지친 베스가 비좁은 아파트에서 세 아이를 키우는 엘리를 뒤늦게 찾아간다. 아파트에서 신비로운 책이 발견되며 육체에 사로잡힌 악마가 발견되며, 베스가 가장 악몽 같은 버전의 모성애에 직면하게 되면서 생존에 위한 원초적인 싸움에 내몰리게 되는데...',
  'popularity': 5030.612,
  'poster_path': '/5ik4ATKmNtmJU6AYD0bLm56BCVM.jpg',
  'release_date': '2023-04-12',
  'title': '이블 데드 라이즈',
  'video': False,
  'vote_average': 7.1,
  'vote_count': 1017},
 {'adult': False,
  'backdrop_path': '/3CxUndGhUcZdt1Zggjdb2HkLLQX.jpg',
  'genre_ids': [28, 12, 878],
  'id': 640146,
  'original_language': 'en',
  'original_title': 'Ant-Man and the Wasp: Quantumania',
  'overview': '슈퍼히어로 파트너인 스캇 랭과 호프 반 다인, 호프의 부모 재닛 반 다인과 행크 핌, 그리고 스캇의 딸 캐시 랭까지 미지의 양자 영역 세계 속에 빠져버린 앤트맨 패밀리. 그 곳에서 새로운 존재들과 무한한 우주를 다스리는 정복자 캉을 만나며, 그 누구도 예상 못 한 모든 것의 한계를 뛰어넘는 모험을 시작하게 되는데…',
  'popularity': 3023.011,
  'poster_path': '/sz6mTIDDQmR

In [30]:
english_features_name = list(english_movies[0].keys())

In [31]:
features_name = list(movies[0].keys())

In [32]:
df_english_movies = pd.DataFrame(english_movies, columns= english_features_name)
df_english_movies.head(3)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/aAgGrfBwna1nO4M2USxwFgK5O0t.jpg,"[27, 53, 14]",713704,en,Evil Dead Rise,Three siblings find an ancient vinyl that give...,5030.612,/5ik4ATKmNtmJU6AYD0bLm56BCVM.jpg,2023-04-12,Evil Dead Rise,False,7.1,1017
1,False,/3CxUndGhUcZdt1Zggjdb2HkLLQX.jpg,"[28, 12, 878]",640146,en,Ant-Man and the Wasp: Quantumania,Super-Hero partners Scott Lang and Hope van Dy...,3023.011,/qnqGbB22YJ7dSs4o6M7exTpNxPz.jpg,2023-02-15,Ant-Man and the Wasp: Quantumania,False,6.5,2525
2,False,/nLBRD7UPR6GjmWQp6ASAfCTaWKX.jpg,"[16, 10751, 12, 14, 35]",502356,en,The Super Mario Bros. Movie,"While working underground to fix a water main,...",4675.159,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,2023-04-05,The Super Mario Bros. Movie,False,7.5,2202


In [33]:
df_movies = pd.DataFrame(movies, columns= features_name)
df_movies.head(3)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/aAgGrfBwna1nO4M2USxwFgK5O0t.jpg,"[27, 53, 14]",713704,en,Evil Dead Rise,길바닥에서 지친 베스가 비좁은 아파트에서 세 아이를 키우는 엘리를 뒤늦게 찾아간다....,5030.612,/5ik4ATKmNtmJU6AYD0bLm56BCVM.jpg,2023-04-12,이블 데드 라이즈,False,7.1,1017
1,False,/3CxUndGhUcZdt1Zggjdb2HkLLQX.jpg,"[28, 12, 878]",640146,en,Ant-Man and the Wasp: Quantumania,"슈퍼히어로 파트너인 스캇 랭과 호프 반 다인, 호프의 부모 재닛 반 다인과 행크 핌...",3023.011,/sz6mTIDDQmR3DYgJudiTmoW2gR5.jpg,2023-02-15,앤트맨과 와스프: 퀀텀매니아,False,6.5,2525
2,False,/nLBRD7UPR6GjmWQp6ASAfCTaWKX.jpg,"[16, 10751, 12, 14, 35]",502356,en,The Super Mario Bros. Movie,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,4675.159,/dlGyzCxbBQK1U2O5o31Z1hB6erc.jpg,2023-04-05,슈퍼 마리오 브라더스,False,7.5,2202


In [34]:
df_movies = df_movies.drop(columns=['backdrop_path'])
df_movies.head(3)

,adult,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,"[27, 53, 14]",713704,en,Evil Dead Rise,길바닥에서 지친 베스가 비좁은 아파트에서 세 아이를 키우는 엘리를 뒤늦게 찾아간다....,5030.612,/5ik4ATKmNtmJU6AYD0bLm56BCVM.jpg,2023-04-12,이블 데드 라이즈,False,7.1,1017
1,False,"[28, 12, 878]",640146,en,Ant-Man and the Wasp: Quantumania,"슈퍼히어로 파트너인 스캇 랭과 호프 반 다인, 호프의 부모 재닛 반 다인과 행크 핌...",3023.011,/sz6mTIDDQmR3DYgJudiTmoW2gR5.jpg,2023-02-15,앤트맨과 와스프: 퀀텀매니아,False,6.5,2525
2,False,"[16, 10751, 12, 14, 35]",502356,en,The Super Mario Bros. Movie,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,4675.159,/dlGyzCxbBQK1U2O5o31Z1hB6erc.jpg,2023-04-05,슈퍼 마리오 브라더스,False,7.5,2202


In [35]:
df_movies['eng_title'] = df_english_movies['title']

In [36]:
df_movies = df_movies.drop(columns=['original_title'])
df_movies.head()

,adult,genre_ids,id,original_language,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,eng_title
0,False,"[27, 53, 14]",713704,en,길바닥에서 지친 베스가 비좁은 아파트에서 세 아이를 키우는 엘리를 뒤늦게 찾아간다....,5030.612,/5ik4ATKmNtmJU6AYD0bLm56BCVM.jpg,2023-04-12,이블 데드 라이즈,False,7.1,1017,Evil Dead Rise
1,False,"[28, 12, 878]",640146,en,"슈퍼히어로 파트너인 스캇 랭과 호프 반 다인, 호프의 부모 재닛 반 다인과 행크 핌...",3023.011,/sz6mTIDDQmR3DYgJudiTmoW2gR5.jpg,2023-02-15,앤트맨과 와스프: 퀀텀매니아,False,6.5,2525,Ant-Man and the Wasp: Quantumania
2,False,"[16, 10751, 12, 14, 35]",502356,en,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,4675.159,/dlGyzCxbBQK1U2O5o31Z1hB6erc.jpg,2023-04-05,슈퍼 마리오 브라더스,False,7.5,2202,The Super Mario Bros. Movie
3,False,"[27, 53, 14]",758323,en,바티칸이 인정한 공식 수석 엑소시스트이자 최고의 구마사제 '가브리엘 아모르트’ 신부...,2378.680,/fA1xRPplBqW87wWh0hO1pwbQmX8.jpg,2023-04-05,엑소시스트: 더 바티칸,False,7.2,971,The Pope's Exorcist
4,False,"[878, 12, 28]",447365,en,가모라를 잃고 슬픔에 빠져 있던 피터 퀼이 위기에 처한 은하계와 동료를 지키기 위해...,2137.214,/b7epV2cQZVIR4u5VZraDwD0AgiE.jpg,2023-05-03,가디언즈 오브 갤럭시: Volume 3,False,8.2,1200,Guardians of the Galaxy Vol. 3


In [37]:
# 장르 가져오기 https://api.themoviedb.org/3/genre/movie/list?api_key=<<api_key>>
genre_url = 'https://api.themoviedb.org/3/genre/movie/list?api_key=' + my_api_key
genre_res = requests.get(genre_url).text
genre_list = json.loads(genre_res)['genres']

In [38]:
genre_list

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 80, 'name': 'Crime'},
 {'id': 99, 'name': 'Documentary'},
 {'id': 18, 'name': 'Drama'},
 {'id': 10751, 'name': 'Family'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 36, 'name': 'History'},
 {'id': 27, 'name': 'Horror'},
 {'id': 10402, 'name': 'Music'},
 {'id': 9648, 'name': 'Mystery'},
 {'id': 10749, 'name': 'Romance'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 10770, 'name': 'TV Movie'},
 {'id': 53, 'name': 'Thriller'},
 {'id': 10752, 'name': 'War'},
 {'id': 37, 'name': 'Western'}]

In [39]:
df_genres = pd.DataFrame(genre_list, columns=['id', 'name'])
df_genres.head()

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime


In [40]:
genre_names = []
for idx in range(len(df_movies)):
    movie = df_movies.iloc[idx]
    change_list = []
    for gen in movie['genre_ids']:
        # print(list(df_genres['name'][df_genres['id'] == gen]))
        change_list.append(list(df_genres['name'][df_genres['id'] == gen])[0])
    # print(change_list)
    genre_names.append(change_list)
    # movie['genre_ids'] = change_list
    # print(movie)
df_movies['genre_ids'] = genre_names
df_movies.head()

,adult,genre_ids,id,original_language,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,eng_title
0,False,"[Horror, Thriller, Fantasy]",713704,en,길바닥에서 지친 베스가 비좁은 아파트에서 세 아이를 키우는 엘리를 뒤늦게 찾아간다....,5030.612,/5ik4ATKmNtmJU6AYD0bLm56BCVM.jpg,2023-04-12,이블 데드 라이즈,False,7.1,1017,Evil Dead Rise
1,False,"[Action, Adventure, Science Fiction]",640146,en,"슈퍼히어로 파트너인 스캇 랭과 호프 반 다인, 호프의 부모 재닛 반 다인과 행크 핌...",3023.011,/sz6mTIDDQmR3DYgJudiTmoW2gR5.jpg,2023-02-15,앤트맨과 와스프: 퀀텀매니아,False,6.5,2525,Ant-Man and the Wasp: Quantumania
2,False,"[Animation, Family, Adventure, Fantasy, Comedy]",502356,en,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,4675.159,/dlGyzCxbBQK1U2O5o31Z1hB6erc.jpg,2023-04-05,슈퍼 마리오 브라더스,False,7.5,2202,The Super Mario Bros. Movie
3,False,"[Horror, Thriller, Fantasy]",758323,en,바티칸이 인정한 공식 수석 엑소시스트이자 최고의 구마사제 '가브리엘 아모르트’ 신부...,2378.680,/fA1xRPplBqW87wWh0hO1pwbQmX8.jpg,2023-04-05,엑소시스트: 더 바티칸,False,7.2,971,The Pope's Exorcist
4,False,"[Science Fiction, Adventure, Action]",447365,en,가모라를 잃고 슬픔에 빠져 있던 피터 퀼이 위기에 처한 은하계와 동료를 지키기 위해...,2137.214,/b7epV2cQZVIR4u5VZraDwD0AgiE.jpg,2023-05-03,가디언즈 오브 갤럭시: Volume 3,False,8.2,1200,Guardians of the Galaxy Vol. 3


In [42]:
# image_url = 'https://image.tmdb.org/t/p/original/' + poster_path
# image_url = 'https://image.tmdb.org/t/p/original/' + df_movies.iloc[영화번호]['poster_path']
image_url

NameError: name 'image_url' is not defined

In [101]:
! pip install datetime

     ---------------------------------------- 52.1/52.1 KB ? eta 0:00:00
     ------------------------------------- 204.1/204.1 KB 12.9 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [98]:
df_today_movies.to_json(r'C:\Users\SSAFY\Desktop\1학기\PJT\final_pjt\back-end\movies\fixture\today_movie.json')

In [75]:
df_today_movies

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/A7JQ7MIV5fkIxceI5hizRIe6DRJ.jpg,"[878, 12, 28]",447365,en,Guardians of the Galaxy Vol. 3,가모라를 잃고 슬픔에 빠져 있던 피터 퀼이 위기에 처한 은하계와 동료를 지키기 위해...,2137.214,/b7epV2cQZVIR4u5VZraDwD0AgiE.jpg,2023-05-03,가디언즈 오브 갤럭시: Volume 3,False,8.164,1207
1,False,/nLBRD7UPR6GjmWQp6ASAfCTaWKX.jpg,"[16, 10751, 12, 14, 35]",502356,en,The Super Mario Bros. Movie,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,4675.159,/dlGyzCxbBQK1U2O5o31Z1hB6erc.jpg,2023-04-05,슈퍼 마리오 브라더스,False,7.513,2246
2,False,/v39H1nUYxQmSphz83VoV4HYjuHt.jpg,"[18, 35]",636173,ko,드림,"선수 생활 사상 최악의 위기를 맞은 쏘울리스 축구 선수 홍대. 계획도, 의지도 없던...",19.450,/9RIY2JW0JLQl2MOXdkeFAVqP57T.jpg,2023-04-26,드림,False,6.000,3
3,False,/zlHeQg9OTYED2Gm4ZErdvElsUFg.jpg,"[16, 35]",955666,ja,映画クレヨンしんちゃん もののけニンジャ珍風伝,"어느 날 ‘짱구‘와 동갑내기인 5살 ‘진구’를 데리고, 짱구 가족을 찾아온 수상한 ...",5.634,/4DKjTt6VodfMRZ1Xs77peiQQXmY.jpg,2022-04-22,짱구는 못말려 극장판: 동물소환 닌자 배꼽수비대,False,7.700,6
4,False,/h8gHn0OzBoaefsYseUByqsmEDMY.jpg,"[28, 53, 80]",603692,en,John Wick: Chapter 4,죽을 위기에서 살아난 존 윅은 최고 회의를 쓰러트릴 방법을 찾아낸다. 비로소 완전한...,1175.963,/9WF6TxCYwdiZw51NM92ConaQz1w.jpg,2023-03-22,존 윅 4,False,7.913,1403
5,False,/cEyhk8tZWubni71M6plwLMQFOIX.jpg,"[28, 80]",385687,en,Fast X,돔과 그의 패밀리 앞에 나타난 운명의 적 단테. 과거의 그림자는 돔의 모든 것을 파...,1623.730,/3JT1oE0JIA6CQJaZDG00y3S0kP3.jpg,2023-05-17,분노의 질주: 라이드 오어 다이,False,6.200,29
6,False,/3MXFMSRHuRMidknZ5CpFI5oLcY6.jpg,[99],1111319,ko,문재인입니다,대한민국 제19대 대통령으로 5년의 임기를 마치고 퇴임한 문재인 전 대통령은 평산마...,6.378,/e8WjLf0a0otxNj1oUhw7FlrUwZr.jpg,2023-05-10,문재인입니다,False,6.000,1
7,False,/ceYZCBfwbBwSpGJ6PapNVw5jqLG.jpg,"[16, 18, 12, 14]",916224,ja,すずめの戸締まり,"큐슈의 작은 마을에서 살고 있는 소녀 ‘스즈메’는 어느 날, 여행을 하며 ‘문’을 ...",128.588,/kVS51ssZF1y0IXF342h54cIJ0EK.jpg,2022-11-11,스즈메의 문단속,False,7.938,363
8,False,None,"[10749, 35]",840612,ko,롱디,인디 밴드 연신굽신의 열렬한 팬에서 밴드의 보컬 태인과 연인으로 발전해 5년째 연애...,6.539,/cBRY4UrbKcaYA3wawkXuTWYKnPN.jpg,2023-05-10,롱디,False,9.000,1
9,False,/zjpYDQlhvrAaohKRShu522sKJ87.jpg,"[16, 35, 18]",783675,ja,THE FIRST SLAM DUNK,"전국 제패를 꿈꾸는 북산고 농구부 5인방의 꿈과 열정, 멈추지 않는 도전을 그린 영화",49.787,/coiJrdXAXuBkSGDvp9bZ7mkuU6E.jpg,2022-12-03,더 퍼스트 슬램덩크,False,8.067,75


In [44]:
today_movies

[{'rnum': '1',
  'rank': '1',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20231496',
  'movieNm': '가디언즈 오브 갤럭시: Volume 3',
  'openDt': '2023-05-03',
  'salesAmt': '659107901',
  'salesShare': '54.5',
  'salesInten': '-125993518',
  'salesChange': '-16',
  'salesAcc': '30591247548',
  'audiCnt': '61313',
  'audiInten': '-13387',
  'audiChange': '-17.9',
  'audiAcc': '2865085',
  'scrnCnt': '1673',
  'showCnt': '5762'},
 {'rnum': '2',
  'rank': '2',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20227890',
  'movieNm': '슈퍼 마리오 브라더스',
  'openDt': '2023-04-26',
  'salesAmt': '103564361',
  'salesShare': '8.6',
  'salesInten': '-15717155',
  'salesChange': '-13.2',
  'salesAcc': '20535101068',
  'audiCnt': '10155',
  'audiInten': '-1735',
  'audiChange': '-14.6',
  'audiAcc': '2085093',
  'scrnCnt': '683',
  'showCnt': '1449'},
 {'rnum': '3',
  'rank': '3',
  'rankInten': '1',
  'rankOldAndNew': 'OLD',
  'movieCd': '20198482',
  'movieNm': '드림',
  'openDt': '202

In [50]:
today_movie_detail_list

[{'adult': False,
  'backdrop_path': '/A7JQ7MIV5fkIxceI5hizRIe6DRJ.jpg',
  'genre_ids': [878, 12, 28],
  'id': 447365,
  'original_language': 'en',
  'original_title': 'Guardians of the Galaxy Vol. 3',
  'overview': '가모라를 잃고 슬픔에 빠져 있던 피터 퀼이 위기에 처한 은하계와 동료를 지키기 위해 다시 한 번 가디언즈 팀과 힘을 모으고, 성공하지 못할 경우 그들의 마지막이 될지도 모르는 미션에 나서는 이야기.',
  'popularity': 2137.214,
  'poster_path': '/b7epV2cQZVIR4u5VZraDwD0AgiE.jpg',
  'release_date': '2023-05-03',
  'title': '가디언즈 오브 갤럭시: Volume 3',
  'video': False,
  'vote_average': 8.164,
  'vote_count': 1207},
 {'adult': False,
  'backdrop_path': '/nLBRD7UPR6GjmWQp6ASAfCTaWKX.jpg',
  'genre_ids': [16, 10751, 12, 14, 35],
  'id': 502356,
  'original_language': 'en',
  'original_title': 'The Super Mario Bros. Movie',
  'overview': "따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평범한 배관공 형제 '마리오'와 ‘루이지’는 배수관 고장으로 위기에 빠진 도시를 구하려다 미스터리한 초록색 파이프 안으로 빨려 들어가게 된다. 파이프를 통해 새로운 세상으로 차원 이동하게 된 형제. 형 '마리오'는 뛰어난 리더십을 지닌 '피치'가 통치하는 버섯왕국에 도착하지만 동생 '루이지'는 빌런 '쿠파'가 있는 다크랜드로 떨어지며 

In [76]:
df_today_movies.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/A7JQ7MIV5fkIxceI5hizRIe6DRJ.jpg,"[Science Fiction, Adventure, Action]",447365,en,Guardians of the Galaxy Vol. 3,가모라를 잃고 슬픔에 빠져 있던 피터 퀼이 위기에 처한 은하계와 동료를 지키기 위해...,2137.214,/b7epV2cQZVIR4u5VZraDwD0AgiE.jpg,2023-05-03,가디언즈 오브 갤럭시: Volume 3,False,8.164,1207
1,False,/nLBRD7UPR6GjmWQp6ASAfCTaWKX.jpg,"[Animation, Family, Adventure, Fantasy, Comedy]",502356,en,The Super Mario Bros. Movie,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,4675.159,/dlGyzCxbBQK1U2O5o31Z1hB6erc.jpg,2023-04-05,슈퍼 마리오 브라더스,False,7.513,2246
2,False,/v39H1nUYxQmSphz83VoV4HYjuHt.jpg,"[Drama, Comedy]",636173,ko,드림,"선수 생활 사상 최악의 위기를 맞은 쏘울리스 축구 선수 홍대. 계획도, 의지도 없던...",19.450,/9RIY2JW0JLQl2MOXdkeFAVqP57T.jpg,2023-04-26,드림,False,6.000,3
3,False,/zlHeQg9OTYED2Gm4ZErdvElsUFg.jpg,"[Animation, Comedy]",955666,ja,映画クレヨンしんちゃん もののけニンジャ珍風伝,"어느 날 ‘짱구‘와 동갑내기인 5살 ‘진구’를 데리고, 짱구 가족을 찾아온 수상한 ...",5.634,/4DKjTt6VodfMRZ1Xs77peiQQXmY.jpg,2022-04-22,짱구는 못말려 극장판: 동물소환 닌자 배꼽수비대,False,7.700,6
4,False,/h8gHn0OzBoaefsYseUByqsmEDMY.jpg,"[Action, Thriller, Crime]",603692,en,John Wick: Chapter 4,죽을 위기에서 살아난 존 윅은 최고 회의를 쓰러트릴 방법을 찾아낸다. 비로소 완전한...,1175.963,/9WF6TxCYwdiZw51NM92ConaQz1w.jpg,2023-03-22,존 윅 4,False,7.913,1403


In [74]:
new_poster_url = 
for today_movie_detail in today_movie_detail_list:
    print(today_movie_detail['id'])
    print(today_movie_detail['title'])
    poster_url = 'https://image.tmdb.org/t/p/original/' + today_movie_detail['poster_path']
    poster_res = requests.get(poster_url)
    poster_img = Image.open(BytesIO(poster_res.content))
    print(poster_img)
    
    poster_img.show()

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12252\2943830785.py:4: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  urllib._urlopener = AppURLopener()


447365
가디언즈 오브 갤럭시: Volume 3
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1080x1620 at 0x2C3812D1970>
502356
슈퍼 마리오 브라더스
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2000x3000 at 0x2C381187E80>
636173
드림
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1429x2143 at 0x2C38117BA90>
955666
짱구는 못말려 극장판: 동물소환 닌자 배꼽수비대
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2000x3000 at 0x2C381187E80>
603692
존 윅 4
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1900x2850 at 0x2C38117B520>
385687
분노의 질주: 라이드 오어 다이
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1890x2835 at 0x2C3812D1F10>
1111319
문재인입니다
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1980x2835 at 0x2C3812D1730>
916224
스즈메의 문단속
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1900x2835 at 0x2C381181340>
840612
롱디
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=900x1350 at 0x2C381187EE0>
783675
더 퍼스트 슬램덩크
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x2399 

In [85]:
# 유튜브 video url
# 'https://www.youtube.com/watch?v=' + video에 있는 key

# 연관 video url
movie_id = '447365'
video_url = 'https://api.themoviedb.org/3/movie/' + movie_id + '/videos?api_key=' + my_api_key +'&language=en-US'
video_res = requests.get(video_url).text
video_data = json.loads(video_res)
video_data = video_data['results'][:5]

yt_url_list = []

for video_key in video_data:
    video_key = video_key['key']
    yt_url = 'https://www.youtube.com/watch?v=' + video_key
    yt_url_list.append(yt_url)

df_today_movies['video'] = yt_url_list
df_today_movies

ValueError: Length of values (5) does not match length of index (10)

# 데이터 score화

In [6]:
with open('./movies/fixtures/AllMovie.json') as f:
    all_movie_js = json.loads(f.read()) ## json 라이브러리 이용

all_movie_list = list(all_movie_js)

df = pd.DataFrame(all_movie_list, columns=list(all_movie_list[0].keys()))
df.head(3)

,adult,backdrop_path,movie_id,original_language,overview,popularity,poster_path,release_date,title,vote_average,vote_count,eng_title
0,False,/aAgGrfBwna1nO4M2USxwFgK5O0t.jpg,713704,en,길바닥에서 지친 베스가 비좁은 아파트에서 세 아이를 키우는 엘리를 뒤늦게 찾아간다....,5030.612,/5ik4ATKmNtmJU6AYD0bLm56BCVM.jpg,2023-04-12,이블 데드 라이즈,7.1,1017,Evil Dead Rise
1,False,/3CxUndGhUcZdt1Zggjdb2HkLLQX.jpg,640146,en,"슈퍼히어로 파트너인 스캇 랭과 호프 반 다인, 호프의 부모 재닛 반 다인과 행크 핌...",3023.011,/sz6mTIDDQmR3DYgJudiTmoW2gR5.jpg,2023-02-15,앤트맨과 와스프: 퀀텀매니아,6.5,2525,Ant-Man and the Wasp: Quantumania
2,False,/nLBRD7UPR6GjmWQp6ASAfCTaWKX.jpg,502356,en,따단-딴-따단-딴 ♫ 전 세계를 열광시킬 올 타임 슈퍼 어드벤처의 등장! 뉴욕의 평...,4675.159,/dlGyzCxbBQK1U2O5o31Z1hB6erc.jpg,2023-04-05,슈퍼 마리오 브라더스,7.5,2202,The Super Mario Bros. Movie


In [7]:
df.describe()

,movie_id,popularity,vote_average,vote_count
count,9.712000e+03,9712.000000,9712.00000,9712.000000
mean,2.890043e+05,29.589584,6.41110,1587.514003
std,3.226298e+05,102.715181,1.23986,2898.962197
min,5.000000e+00,7.152000,0.00000,0.000000
25%,1.156300e+04,13.177750,6.00000,158.000000
50%,1.197180e+05,16.801500,6.60000,543.000000
75%,5.206772e+05,25.240500,7.20000,1618.250000
max,1.123698e+06,5030.612000,10.00000,33679.000000


In [4]:
def preprocess(movie_name):
    import numpy as np
    from numpy import dot
    from numpy.linalg import norm
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    import json

    with open('./movies/fixtures/AllMovie.json', 'r', encoding='UTF-8') as f:
        today_movie_js = json.loads(f.read()) ## json 라이브러리 이용

    # DataFrame 생성
    df = pd.DataFrame(today_movie_js)

    # tfidf를 통하여 data 변환
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['overview'])

    # cosine_similarity matrix 생성
    cosine_similar = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # title : index로 dictionary 생성
    cs_idx = dict(zip(df['title'], df.index))

    def get_recommendations(title, cosine_similar=cosine_similar):
        # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받옴
        idx = cs_idx[title]

        # 해당 영화와 모든 영화와의 유사도를 가져옴
        sim_scores = list(enumerate(cosine_similar[idx]))

        # 유사도에 따라 영화들을 정렬
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 가장 유사한 10개의 영화를 받아옴
        sim_scores = sim_scores[1:11]

        # 가장 유사한 10개의 영화의 인덱스를 얻음
        movie_indices = [idx[0] for idx in sim_scores]

        # 가장 유사한 10개의 영화의 제목을 리턴
        return df['title'].iloc[movie_indices]
    
    return_value = list(get_recommendations(movie_name))
    
    return return_value

df = preprocess('어벤져스')
df
# print(len(df))
# recommend_list = []
# feature_name = df.columns
# for idx in range(len(df)):
#     recommend_list.append(
#         dict(df.iloc[idx])
#     )
# recommend_list

['어벤져스 컨피덴셜: 블랙 위도우 앤 퍼니셔',
 '캡틴 아메리카: 윈터 솔져',
 '캡틴 아메리카: 시빌 워',
 '닥터 스트레인지: 대혼돈의 멀티버스',
 '오토라는 남자',
 '어벤져스: 인피니티 워',
 '어벤져스: 에이지 오브 울트론',
 '트랜스포머',
 '드래곤 네스트: 평화 기사단 VS 블랙 드래곤',
 '스트롬볼리']